In [1]:
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split, cross_validate, KFold
import seaborn as sns
from lightgbm.sklearn import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score, accuracy_score
from sklearn.metrics import make_scorer

from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet
from sklearn.naive_bayes import BernoulliNB

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize.casual import TweetTokenizer
from hashlib import md5
import numpy as np

%matplotlib notebook

mse = make_scorer(mse)
mae = make_scorer(mae)
r2_score = make_scorer(r2_score)
accuracy_score = make_scorer(accuracy_score)

In [2]:
from mlxtend.regressor import StackingRegressor, StackingCVRegressor
from mlxtend.classifier import StackingClassifier

In [7]:
data = pd.read_csv("X_train.csv")

def preprocess_data(data):
    def func(str_):
        global l
        l = eval(str_)
        dict_ = dict(zip(*
            itertools.chain.from_iterable((x.keys(), x.values()) for x in l)
        ))
        return dict_

    data["property"] = data["property"].apply(lambda str_: dict((list(x.keys())[0], list(x.values())[0]) for x in eval(str_)))
    data["property"] = data["property"].apply(lambda x: " ".join("_".join((str(k), v)) for k, v in x.items()))
    data = pd.get_dummies(data, columns=['brandId', 'categoryLevel1Id', 'categoryLevel2Id'])
    
    data["commentNegative"].fillna("EMPTY", inplace=True)
    data["commentPositive"].fillna("EMPTY", inplace=True)

    if True:
        l = [data]

        for col in ["comment"]:# , "commentNegative", "commentPositive"]:
            for i, vectorizer in enumerate([
                TfidfVectorizer(tokenizer=TweetTokenizer(False).tokenize, ngram_range=(1, 2), min_df=0.01),
                TfidfVectorizer(analyzer="char", ngram_range=(1, 4), min_df=0.01)
            ]):
                count = vectorizer.fit_transform(data[col])

                l.append(
                    pd.DataFrame(
                        count.A,
                        columns=[
                            col + " " + str(i) + " " +
                            word.replace("<", "").replace(">", "").replace("[", "").replace("]", "")
                            for word in vectorizer.get_feature_names()
                        ]
                    )
                )

            data.drop(col, axis=1, inplace=True)
        data = pd.concat(l, axis=1)
    
    target = data['reting']
    data["date"] = pd.to_datetime(data["date"]).astype(int)
    data.drop(['userName', 'reting', 'sku'], inplace=True, axis=1)
    return (data, target)
    
train, test = train_test_split(data, test_size=0.15)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
train_data, train_target = preprocess_data(train)
test_data, test_target = preprocess_data(test)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [8]:
train_data.shape

(13248, 11328)

In [9]:
train_data.drop(["commentNegative", "commentPositive", "property"], axis=1, inplace=True)

In [ ]:
regs = [LGBMRegressor(), XGBRegressor()]
reg = StackingRegressor(regs, XGBRegressor())

scoring = {
    "MSE": mse,
    "MAE": mae,
    "r2_score": r2_score
}

cv = cross_validate(KNeighborsRegressor(25), train_data, train_target, scoring=scoring, cv=KFold(10), verbose=3)

[CV]  ................................................................
[CV]  , MSE=1.7769891501886796, MAE=1.065467169811321, r2_score=-0.021120861371558552, total=  34.8s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.7min remaining:    0.0s


In [ ]:
cv = {k: (v.mean(), v.std()) for k, v in cv.items()}

In [ ]:
cv

In [ ]:
cv = cross_val_score(SVR(kernel='linear'), train_data, train_target, scoring=mse, cv=KFold(10), verbose=3)

In [ ]:
counts = np.array([len(x) for x in data.sort_values("userName").groupby("userName").groups.values()])

In [ ]:
len(counts)

In [ ]:
len(counts[counts < 2])

In [ ]:
sns.distplot(data.groupby('userName').reting.mean())

In [ ]:
[data.loc[x]['reting'].mean() for x in data.groupby("userName").groups.values()]

In [ ]:
data

In [ ]:
data["property"]

In [ ]:
x = data["property"].apply(lambda x: set.union(*[set(a.keys()) for a in eval(x)]))

In [ ]:
sorted(set.union(*x))

In [ ]:
data[data['sku'] == 30029584]

In [ ]:
import hashlib

In [ ]:
import numpy as np

In [ ]:
[
    list(d.keys())[0]
    if
    hashlib.md5("PSP-3008".encode("utf-8")).hexdigest() in d.values()
    else
    False 
    for d in eval(data.loc[14].property)
][26]

In [ ]:
[
    any(2537 in dd.keys() for dd in eval(d[1].property))
    for d in data.iterrows()
]

In [ ]:
len(eval(data.loc[14].property))

In [ ]:
data.sku

In [ ]:
x = data[data['sku'] == data['sku'][0]].property

In [ ]:
x = list(x)

In [ ]:
x[0] == x[3]

In [ ]:
data.loc[13]

In [ ]:
~data["commentNegative"].isnull() & ~data["commentPositive"].isnull()

In [ ]:
data["commentPositive"]